In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# from __future__ import unicode_literals
import numpy as np
from PIL import Image
import time
import argparse
import pyflow

import cv2
import matplotlib.pyplot as plt

import os
import argparse
import numpy as np
import torch.utils.data
from glob import glob
from PIL import Image
from tqdm import tqdm
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import misc
import time
from IPython import display
import gc
import json

In [2]:
def get_dataset(names_dir):
        # get video names
        file = open(names_dir,'r') 
        names = file.readlines()
        
        # get the images
        imgs = []
        maps = []
        for name in names:  
            maps.append(sorted(glob(os.path.join('data/DAVIS/Annotations/480p', name[:-1],  '*'))))
            imgs.append(sorted(glob(os.path.join('data/DAVIS/JPEGImages/480p', name[:-1],  '*'))))
        return imgs, maps
    
train_data = get_dataset('data/DAVIS/ImageSets/2017/train.txt')
val_data = get_dataset('data/DAVIS/ImageSets/2017/val.txt')

train_data_size = 0
val_data_size = 0

for i in range(len(train_data[0])):
    train_data_size += len(train_data[0][i])
    
for i in range(len(val_data[0])):
    val_data_size += len(val_data[0][i])
    
print("train data size = ",train_data_size)
print("val_data_size = ",val_data_size)

train_length = np.zeros(len(train_data[0]),dtype = int)
val_length = np.zeros(len(val_data[0]),dtype = int)

train data size =  4209
val_data_size =  1999


In [3]:
#pre-load dataset to speed up further data preparations
def_h = 480
def_w = 840

train_counter = 0
val_counter = 0
n_channels = 3

Train_data_seg = np.zeros((train_data_size,def_h,def_w), dtype = np.uint8)
Val_data_seg = np.zeros((val_data_size,def_h,def_w), dtype = np.uint8)

Train_data_img = np.zeros((train_data_size,def_h,def_w,n_channels), dtype = np.uint8)
Val_data_img = np.zeros((val_data_size,def_h,def_w,n_channels), dtype = np.uint8)

for i in range(len(train_data[1])):
    #number of frames in this particular video
    train_length[i] = len(train_data[1][i])
    for j in range(train_length[i]):
        seg = np.asarray(Image.open(train_data[1][i][j])).copy()
        img = np.asarray(Image.open(train_data[0][i][j])).copy()
        h,w = seg.shape
        #we want to work only with 480 x 848 images
        if w != def_w:
            step = (w-def_w) // 2
            if (w - def_w) % 2 == 0:
                seg = seg[:,step:w-step]
                img = img[:,step:w-step,:]
            else:
                seg = seg[:,step+1:w-step]
                img = img[:,step+1:w-step,:]
        #binary segmentation
        seg[seg > 0] = 1
        Train_data_seg[train_counter,:,:] = seg  
        Train_data_img[train_counter,:,:,:] = img 
        train_counter += 1

for i in range(len(val_data[1])):
    #number of frames in this particular video
    val_length[i] = len(val_data[1][i])
    for j in range(val_length[i]):
        seg = np.asarray(Image.open(val_data[1][i][j])).copy()
        img = np.asarray(Image.open(val_data[0][i][j])).copy()
        h,w = seg.shape
        #we want to work only with 480 x 848 images
        if w != def_w:
            step = (w-def_w) // 2
            if (w - def_w) % 2 == 0:
                seg = seg[:,step:w-step]
                img = img[:,step:w-step,:]
            else:
                seg = seg[:,step+1:w-step]
                img = img[:,step+1:w-step,:]
        #binary segmentation
        seg[seg > 0] = 1
        Val_data_seg[val_counter,:,:] = seg  
        Val_data_img[val_counter,:,:,:] = img 
        val_counter += 1

In [4]:
train_length_cum = np.zeros(len(train_data[0]),dtype = int)
val_length_cum = np.zeros(len(val_data[0]),dtype = int)

train_length_cum = np.cumsum(train_length) - train_length
val_length_cum = np.cumsum(val_length) - val_length

print(train_length_cum)
print(val_length_cum)

[   0   82  172  247  334  405  485  565  654  717  801  853  913  975 1051
 1076 1162 1245 1309 1400 1480 1560 1640 1715 1775 1843 1923 2023 2088 2161
 2213 2283 2353 2433 2503 2563 2606 2652 2722 2760 2810 2900 2935 3015 3106
 3181 3249 3329 3395 3443 3534 3605 3660 3720 3790 3866 3946 4005 4070 4137]
[   0   69  119  199  283  373  448  488  592  682  742  808  860  910 1000
 1078 1128 1209 1243 1293 1340 1389 1439 1518 1558 1638 1738 1817 1860 1900]


In [5]:
def fast_hist(pred, label, n=2):
    k = (label >= 0) & (label < n)
    return np.bincount(n * label[k] + pred[k], minlength=n ** 2).reshape(n, n)
#         n * label[k].astype(int) + pred[k], minlength=n ** 2).reshape(n, n)

def per_class_iu(hist):
    return np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))

# def compute_metrics(pred,label,n_classes):
#     """
#     pred,label = np.array, shape = (batch_size,H,W), entries \in [0,n_classes-1]
#     """
#     iou = np.zeros(n_classes,dtype = float)
#     accuracy = []
#     for i in range(pred.shape[0]):
#         hist = fast_hist(pred[i,:,:],label[i,:,:],n_classes)
#         iou += per_class_iu(hist)
#         accuracy.append(np.diag(hist).sum()/hist.sum())
#     iou = iou / pred.shape[0]
#     mean_iou = iou.mean()
#     mean_accuracy = np.mean(accuracy)
#     return mean_accuracy,mean_iou,iou

def compute_metrics(pred, label, n_classes):
    """
    pred, label = np.array, shape = (batch_size, H, W), entries \in [0, n_classes - 1]
    """
    iou = np.zeros(n_classes, dtype = float)
    accuracy = []
    hist = fast_hist(pred, label, n_classes)
    iou += per_class_iu(hist)
    accuracy.append(np.diag(hist).sum() / hist.sum())
    iou = iou
    mean_iou = iou.mean()
    mean_accuracy = np.mean(accuracy)
    return mean_accuracy, mean_iou, iou

In [6]:
def generate_batches_AR_all_indices(dataset, cumul, n_frames=1, 
                                    n_out=1, step=1):
    """
    generates start indices
    """
    indices = []
    for vid in range(len(dataset[0])):
        for start in range(0, len(dataset[0][vid]) - n_frames - n_out + 1, step):
            start_ind = cumul[vid] + start
            #end_ind = start_ind + n_frames + n_out
            indices.append(start_ind)
    res = {
        "indices": indices, "n_frames": n_frames, "n_out": n_out
    }    
    return res


def generate_batches_AR_all(dataset, X_seg, indices_dict, batch_size=2):
    start_indices = np.array(indices_dict["indices"])
    n_frames = indices_dict["n_frames"]
    n_out = indices_dict["n_out"]
    random_indices = np.random.permutation(np.arange(len(start_indices)))
    for start in range(0, len(random_indices), batch_size):
        batch_random_indices = random_indices[start: start + batch_size]
        batch_start_indices = start_indices[batch_random_indices]
        X_batch = []
        y_batch = []
        for out in range(n_out):
            loc_X = []
            loc_y = []
            for b in range(batch_size):
                if b >= len(batch_start_indices):
                    break
                start_ind = batch_start_indices[b]
                seg = X_seg[start_ind+out:start_ind+n_frames+out+1,:,:]
        
                #seg = torch.from_numpy(seg).float()
                
                #n,h,w = seg.size()            
              
                loc_X.append(seg[:-1,:,:])
                loc_y.append(seg[-1,:,:])
            X_batch.append(np.stack(loc_X))
            y_batch.append(np.stack(loc_y))
        #note that it is 5-dim tensor, not 4-dim
        yield np.stack(X_batch)[0][0], np.stack(y_batch)[0][0]

In [7]:
Train_data_seg.shape

(4209, 480, 840)

### Go

In [8]:
n_out = 1
n_classes = 2
n_frames = 2

In [9]:
train_indices = generate_batches_AR_all_indices(train_data, train_length_cum, n_frames, n_out)
val_indices = generate_batches_AR_all_indices(val_data, val_length_cum, n_frames, n_out)

In [10]:
batch_size = 1
number_of_batches_train = int(np.ceil(len(train_indices["indices"]) / batch_size))
number_of_batches_val = int(np.ceil(len(val_indices["indices"]) / batch_size))

In [11]:
np.random.seed(666)

acc = []
miou = []
iou = []

for X_batch, labels in tqdm_notebook(generate_batches_AR_all(val_data, Val_data_seg, val_indices, batch_size=batch_size)):
    im1 = X_batch[0]
    im2 = X_batch[1]

    im1 = im1.astype(float) / 1.
    im2 = im2.astype(float) / 1.

    im1 = np.expand_dims(im1, 2)
    im2 = np.expand_dims(im2, 2)

    im1 = np.concatenate((im1, im1, im1), axis=2)
    im2 = np.concatenate((im2, im2, im2), axis=2)

    parser = argparse.ArgumentParser(description='Demo for python wrapper of Coarse2Fine Optical Flow')
    parser.add_argument('-viz', dest='viz', action='store_true', help='Visualize (i.e. save) output of flow.')

    # Flow Options:
    alpha = 0.012
    ratio = 0.75
    minWidth = 20
    nOuterFPIterations = 7
    nInnerFPIterations = 1
    nSORIterations = 30
    colType = 0  # 0 or default:RGB, 1:GRAY (but pass gray image with shape (h,w,1))

    s = time.time()
    u, v, im2W = pyflow.coarse2fine_flow(
        im1, im2, alpha, ratio, minWidth, nOuterFPIterations, nInnerFPIterations,
        nSORIterations, colType)
    flow = np.concatenate((u[..., None], v[..., None]), axis=2)

    hsv = np.zeros(im1.shape, dtype=np.uint8)
    hsv[:, :, 0] = 255
    hsv[:, :, 1] = 255
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    im2W = np.einsum('ijk -> kij', im2W)[0]
    im2W[im2W > 0] = 1.
    im2W = im2W.astype(np.int32)

    acc_loc, miou_loc, iou_loc = compute_metrics(im2W, labels, 2)
    
    acc.append(acc_loc)
    miou.append(miou_loc)
    iou.append(iou_loc)

In [12]:
def save_files(acc, miou, iou, path):
    d = {
        "acc": [float(x) for x in acc],
        "iou": [float(y) for x in iou for y in x],
        "miou": [float(x) for x in miou]
    }
    with open(path, 'w') as f:
        json.dump(d, f)

In [13]:
path = 'saved_files/s2s_of.json'
save_files(acc, miou, iou, path)

In [16]:
mean_acc = [np.mean(acc)]
mean_miou = [np.mean(miou)]
mean_iou = [np.mean(iou, axis=0)]

In [17]:
path = 'saved_files/s2s_of_mean.json'
save_files(mean_acc, mean_miou, mean_iou, path)